<a href="https://colab.research.google.com/github/valkryhx/Firefly/blob/master/gptq_lora_baichuan7b_vicuna_v2_%E6%94%AF%E6%8C%81%E5%88%86%E5%B8%83%E5%BC%8F%E8%AE%AD%E7%BB%83_%E5%92%8C%E7%9B%AE%E5%BD%95%E4%BD%9C%E4%B8%BA%E6%95%B0%E6%8D%AE%E9%9B%86_%E8%A7%A3%E5%86%B3%E4%BA%86type%E4%B8%BAbool%E7%9A%84argument%E7%9A%84bug_20230723.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/valkryhx/Firefly

Cloning into 'Firefly'...
remote: Enumerating objects: 750, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 750 (delta 64), reused 28 (delta 28), pack-reused 606
Receiving objects: 100% (750/750), 6.41 MiB | 8.11 MiB/s, done.
Resolving deltas: 100% (432/432), done.


In [ ]:
%cd Firefly/
!pip install -r requirements.txt

/content/Firefly/Firefly
  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-5472a192
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-5472a192
  Resolved https://github.com/huggingface/peft.git to commit e06d94ddeb6c70913593740618df76908b918d66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
pwd

'/content/Firefly'

In [ ]:
#%cd Firefly/

# gptq + lora 训练： 从base model开始  
# 注意 warmup_steps 优先级高 会覆盖warmup_ratio的效果
https://huggingface.co/docs/transformers/main_classes/trainer#transformersTrainingArguments.set_lr_scheduler.warmup_steps

这是train_args/gptq_lora/baichuan7b_gptq_lora.json设置"lr_scheduler_type": "constant_with_warmup"的效果  

In [ ]:
!git pull --all --force
! python train_gptq_lora.py  --train_args_file train_args/gptq_lora/baichuan7b_gptq_lora.json \
                 --output_dir  output/baichuan-gptq-v1 \
                 --use_safetensors False \
                 --model_name_or_path  fireballoon/baichuan-vicuna-chinese-7b-gptq

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), 1.79 KiB | 1.79 MiB/s, done.
From https://github.com/valkryhx/Firefly
   b3123aa..db8cc39  master     -> origin/master
Updating b3123aa..db8cc39
Fast-forward
 train_args/gptq_lora/baichuan7b_gptq_lora.json | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)
[2023-07-23 12:36:15,838] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-23 12:36:21.060042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-23 12:36:25.474 | INFO     | __main__:setup_everything:125 - args.ddp_find_unused_parameters=True,type=<class 'bool'>
2023-07-23 12:36:25.474 | INFO     | __main__:setup_everything:126 - args.use_safetensors=False,type=<class 'bool'>
2023-07-23 12:36:

# gptq + lora 继续训练：从base model挂载上次的lora adapter 继续lora训练

# 注意代码中--ddp_find_unused_parameters的传参方式
#  在deepspeed和torchrun 做分布式训练时 必须让这个参数为False 要显式的指明为False
否则报错 这个报错能查到就是这个参数的锅
RuntimeError: Expected to mark a variable ready only once. This error is caused
by one of the following reasons: 1) Use of a module parameter outside the
`forward` function. Please make sure model parameters are not shared across
multiple concurrent forward-backward passes. or try to use _set_static_graph()
as a workaround if this module graph does not change during training loop.2)
Reused parameters in multiple reentrant backward passes. For example, if you use
multiple `checkpoint` functions to wrap the same part of your model, it would
result in the same set of parameters been used by different reentrant backward
passes multiple times, and hence marking a variable ready multiple times. DDP
does not support such use cases in default. You can try to use


In [ ]:
!git pull --all --force
# !torchrun --nproc-per-node=1
!deepspeed --num_gpus=1 train_gptq_lora.py  --train_args_file train_args/gptq_lora/baichuan7b_gptq_lora.json \
              --peft_path  output/baichuan-gptq-v1/final   \
              --output_dir  output/baichuan-gptq-v2 \
              --ddp_find_unused_parameters False \
              --use_safetensors False \
              --model_name_or_path  fireballoon/baichuan-vicuna-chinese-7b-gptq

Fetching origin
Already up to date.
[2023-07-23 13:42:13,550] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-23 13:42:17.303585: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
[2023-07-23 13:42:18,641] [WARNING] [runner.py:196:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-07-23 13:42:18,655] [INFO] [runner.py:555:main] cmd = /usr/bin/python3 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None train_gptq_lora.py --train_args_file train_args/gptq_lora/baichuan7b_gptq_lora.json --peft_path output/baichuan-gptq-v1/final --output_dir output/baichuan-gptq-v2 --ddp_find_unused_parameters False --use_safetensors False --model_name_or_path fireballoon/baichuan-vicuna-chinese-7b-gptq
[2023-07-23 13:42:20,185] [INFO] [real_accelerator.py:110:get_accele

In [ ]:
pwd

'/content/Firefly/Firefly'

# 测试sft后的单轮对话效果

In [ ]:
!git pull --all --force
!python test_gptq_lora_adapter.py \
     --use_safetensors False \
     --peft_path output/baichuan-gptq-v2/final \
     --base_model_name_or_path fireballoon/baichuan-vicuna-chinese-7b-gptq

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 917 bytes | 50.00 KiB/s, done.
From https://github.com/valkryhx/Firefly
   76b0f1f..31a7237  master     -> origin/master
Updating 76b0f1f..31a7237
Fast-forward
 test_gptq_lora_adapter.py | 5 ++++-
 1 file changed, 4 insertions(+), 1 deletion(-)
[2023-07-23 14:36:52,230] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-23 14:36:58.576972: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-23 14:37:02.803 | INFO     | __main__:main:61 - args.use_safetensors= False , <class 'bool'>
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weight

# 多轮对话 感觉user_input 只输入一次会卡住 要第二次输入才能进入While True循环 还有bug 再调试调试

In [ ]:
!git pull --all --force
!python test_multi_trun_chat_gptq_baichuan7b.py \
     --use_safetensors False \
     --peft_path output/baichuan-gptq-v2/final \
     --base_model_name_or_path fireballoon/baichuan-vicuna-chinese-7b-gptq

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 727 bytes | 727.00 KiB/s, done.
From https://github.com/valkryhx/Firefly
   fb8b1d6..2c65c6e  master     -> origin/master
Updating fb8b1d6..2c65c6e
Fast-forward
 test_multi_trun_chat_gptq_baichuan7b.py | 4 ++++
 1 file changed, 4 insertions(+)
[2023-07-23 15:15:15,517] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
2023-07-23 15:15:20.711783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-23 15:15:24.761 | INFO     | __main__:main:58 - args.use_safetensors= False , <class 'bool'>
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
The model weights are not tied. Please use the `tie_weights